In [1]:
import numpy as np
import torch
from tg import potential_game

In [5]:
def potential1(theta, sv, x, other_params = None):
    # potential1 pulls the solution vector x with a quadratic potential towards x[:]=0
    # the weight of the potentials is given by theta[0] and theta[2]
    return -(theta[0]*(x[0])**2 + theta[2]*(x[1])**2).mean()

def potential2(theta, sv, x, other_params = None):
    # potential2 pulls the solution vector x with a quadratic potential towards x[:]=5
    # the weight of the potentials is given by theta[1] and theta[3]
    return  -(theta[1]*(5-x[0])**2 + theta[3]*(5-x[1])**2).mean()

# construct potential game with convex_ndims = 2 (the dimensionality of the x vector) and the respective player utilities
game = potential_game.PotGame( convex_ndims = 2, player_utilities = [potential1,potential2])

# create and initialize theta game parameters vector
theta = torch.tensor([1,1,1,1]).float()
theta.requires_grad = True

optimizer = torch.optim.Adam([theta], lr=5e-2)


desired_y = torch.tensor([5.,1]).float()

def closure():
    # Forward pass: compute predicted y (solution of game with given theta)
    sol = game.solve(theta, None)
    y = sol[2]
    
    # Compute loss with respect to desired y.
    loss = torch.sum((y[-2:]-desired_y)**2)

    print("Loss", loss.item(), "Game parameters theta", theta)

    # do gradient step to adjust game parameters such that game solution y is closer to desired y
    optimizer.zero_grad() # do zero_grad before backward pass, the inner loop of the game solver also accumulates gradients which have to be cleared
    loss.backward()
    return loss

for t in range(100):
    optimizer.step(closure)


Loss 8.5 Game parameters theta tensor([1., 1., 1., 1.], requires_grad=True)
Loss 7.531256198883057 Game parameters theta tensor([0.9500, 1.0500, 1.0500, 0.9500], requires_grad=True)
Loss 6.627073287963867 Game parameters theta tensor([0.9000, 1.0998, 1.0997, 0.9001], requires_grad=True)
Loss 5.7882232666015625 Game parameters theta tensor([0.8500, 1.1493, 1.1489, 0.8502], requires_grad=True)
Loss 5.015007972717285 Game parameters theta tensor([0.8001, 1.1982, 1.1973, 0.8006], requires_grad=True)
Loss 4.307320594787598 Game parameters theta tensor([0.7502, 1.2465, 1.2446, 0.7513], requires_grad=True)
Loss 3.6646108627319336 Game parameters theta tensor([0.7004, 1.2939, 1.2906, 0.7025], requires_grad=True)
Loss 3.0858826637268066 Game parameters theta tensor([0.6507, 1.3404, 1.3349, 0.6544], requires_grad=True)
Loss 2.5697994232177734 Game parameters theta tensor([0.6012, 1.3856, 1.3774, 0.6071], requires_grad=True)
Loss 2.1145811080932617 Game parameters theta tensor([0.5520, 1.4295, 1.